In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 라이브러리, application 데이터 로딩
import numpy as np
import pandas as pd
import time
import gc
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 화면크기설정
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
#디렉토리 리스트 확인
print(os.listdir("../input/home-credit-default-risk"))

# application_train, application_train data resd_csv

In [ ]:
# application_train resd_csv
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
# application_test resd_csv
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Training data shape: ', app_test.shape)
app_test.head()

In [ ]:
# application_train과 application_test 비교
print('Training data shape: ', app_train.shape)
print('Training data shape: ', app_test.shape)

#target이 빠졌기 때문에 test data가 121개로 한개 적음

In [ ]:
# application_train column 확인
app_train.columns.values

# EDA

TARGET: 예측값

0: 상환 가능함

1: 상환 어려움

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist()

# 대출을 상환가능한 0이 1보다 훨씬 많은 imbalanced data임을 확인

In [ ]:
# 각 column별 결측지 개수 및 비중 확인 함수

def missing_values_table(df):
    # 전체 결측치 개수 확인
    mis_val=df.isnull().sum()
    
    # 결측치 비중 확인
    mis_val_percent=100*df.isnull().sum()/len(df)
    
    # 결측치 개수 , 결측치 비중 테이블 만들기
    mis_val_table=pd.concat([mis_val, mis_val_percent],axis=1)
    
    # 컬럼 이름바꾸기
    mis_val_table_ren_columns=mis_val_table.rename(columns={0:'Missing Values',1:'% of Total Values'})

    # 결측치 0인 컬럼은 제외하고 정렬
    mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=0].sort_values('% of Total Values',ascending=False).round(1)

    # 요약 결과 print
    print("app_train의 전체 컬럼 개수는 "+str(df.shape[1])+"개 이다.\n"
         "그 중에서 결측치가 있는 컬럼 개수는 "+str(mis_val_table_ren_columns.shape[0])+'개 이다.')
    
    return mis_val_table_ren_columns

In [ ]:
missing_values=missing_values_table(app_train)
missing_values.head(20)

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
# 범주형 변수 분석
app_train.select_dtypes('object').nunique()

In [ ]:
# object 칼럼 target값 비교
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
sns.catplot(x='CODE_GENDER', col='TARGET', data=app_train, kind='count')

In [ ]:
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, object_columns)

In [ ]:
# DAYS_BIRTH 컬럼에서는 이상치 없어보임
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
# DAYS_EMPLOYED는 이상치 발견
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment');

In [ ]:
# 이상치 고객 집중 분석
# 이상치
anom=app_train[app_train['DAYS_EMPLOYED']==365243]
# 이상치 외
non_anom=app_train[app_train['DAYS_EMPLOYED']!=365243]

print('The non-anomalies default on %0.2f%% of loans' %(100*non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

# 이상치로 보이는 고객들이 대출을 상환하지못할 확률이 5.4%로 더 낮다.

In [ ]:
# Create an anomalous flag column
## 이상치(365243)인 값에 대해서 True , False로 구분
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# 이상치를 nan값으로 대치
app_train['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)

app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
# test 데이터에도 train 데이터와 동일하게 작업
app_test['DAYS_EMPLOYED_ANOM']=app_test['DAYS_EMPLOYED']==365243
app_test['DAYS_EMPLOYED'].replace({365243:np.nan}, inplace=True)

# True, False로 되어있는 데이터 sum하면 True인것 개수 카운팅됨.
print('There are %d anomalies in the test data out of %d entries'%(app_test['DAYS_EMPLOYED_ANOM'].sum(), len(app_test)))

In [ ]:
# TARGET 변수와의 상관관계
correlations=app_train.corr()['TARGET'].sort_values()

print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [ ]:
# DAYS_BIRTH 컬럼이 가장 양의 상관성이 높음
# 고객이 나이가 많을 수록 상환할 가능성이 적어지는 것처럼 보이지만 DAYS_BIRTH가 음수여서 영향을 받은 것으로 보이므로 절댓값 변환을 해주어 다시 살펴보아야 할 필요가 있음

In [ ]:
# DAYS_BIRTH의 절대값과 TARGET변수와의 상관계수
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

#절댓값 변환 후 상관관계를 다시 보니 고객의 나이가 많을 수록 상환할 가능성이 높음

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포 확인
plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

#고객의 나이는 고르게 분포되어 있음

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

In [ ]:
# 1의 분포를 보면 나이가 젊을 수록 상환을 못할 확률이 높은 것으로 보임
# 나이별로 그룹을 나눠서 target=1을 자세히 분석하고자 함

In [ ]:
# 최소 20 최대 70으로해서 총 10개로 그룹핑
## 20이상 25미만, 25이상 30미만으로 그룹핑됨. 단 (,)는 포함 [,]는 미포함을 의미
np.linspace(20,70,num=11)

In [ ]:
# 나이 그룹별로 상환비율 체크

age_data=app_train[['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH']=age_data['DAYS_BIRTH']/365

# Bin the age data
age_data['YEARS_BINNED']=pd.cut(age_data['YEARS_BIRTH'],bins=np.linspace(20,70,num=11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize=(8,8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation=75);
plt.xlabel('Age Group (years)');
plt.ylabel('Failur to Reapy(%)')
plt.title('Failure to Repay by Age Group');

In [ ]:
# 젋을수록 대출을 상환하지 못하는 것으로 나타남
# 20-25세, 25-30세 30-35세는 각각 약10% 이상 상환 못함
# 55-60세, 60-65세, 65-70세는 5%이하인 것으로 나타남

In [ ]:
#음의 상관이 가장 높았던 3개의 변수 EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3에 대한 분석
ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

# EXT_SOURCE와 TARGET 변수는 음의 상관성 -> EXT_SOURCE값이 증가할수록 상환을 잘하는 것으로 보임

In [ ]:
plt.figure(figsize=(10,12))
plt.subplot(3,1,1);
sns.kdeplot(app_train.loc[app_train['TARGET']==0, 'EXT_SOURCE_1'],label='target==0')
sns.kdeplot(app_train.loc[app_train['TARGET']==1, 'EXT_SOURCE_1'],label='target==1');

In [ ]:
plt.figure(figsize=(10,12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)

In [ ]:
# EXT_SOURCE_3는 target에 따라 차이가 가장 큼

# FE

In [ ]:
apps = pd.concat([app_train, app_test])
print(apps.shape)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].head()

In [ ]:
#컬럼 방향으로 평균을 구하기 위해서 axis=1 추가
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
# 레코드단 mean과 std 만 구함
# max, min 도 해볼 수 있지만, 상관관계가 높은 것에 대해 너무 가공하면 성능이 오히려 떨어질 수 있으므로 mean과 std 만 구함
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
# AMT_CREDIT 비율로 Feature 가공

apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# AMT_CREDIT 비율로 Feature 가공

apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공

# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공. 
apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

In [ ]:
# 데이터 레이블 인코딩, NULL값은 LightGBM 내부에서 처리하도록 특별한 변경하지 않음

object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

In [ ]:
# train data, test data 분리
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

In [ ]:
# data 분리, LGBM Classifier 학습
from sklearn.model_selection import train_test_split

ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

In [ ]:
#test data 예측
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]

In [ ]:
app_test['TARGET'] = preds
app_test[['SK_ID_CURR', 'TARGET']].to_csv('apps_baseline_02.csv', index=False)

참조 code

https://www.kaggle.com/lanakim13/lanasore-ver2

https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction